# Limpieza de los datos de prueba

In [64]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import chardet

import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import os

In [65]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [66]:
nombre_Estacion = input("Ingresar el nombre: ")

Ingresar el nombre: Valle


In [67]:
#Descarga los archivos y los convierte en dataframes

# ID de la carpeta compartida que deseas listar
carpeta_id = '1u9EtFbEvgWRnW1yOwBCVb7Nq8pZvbwpG'

# Lista para almacenar los nombres de los archivos
files_dataframes = []

# Obtener la lista de archivos en la carpeta compartida
file_list = drive.ListFile({'q': "'{}' in parents".format(carpeta_id)}).GetList()

for archivo in file_list:
  if nombre_Estacion.upper() in archivo['title'].upper():
    try:
        # Descargar el archivo y guardarlo en el directorio actual
        archivo.GetContentFile(archivo['title'])
        # Leer el archivo descargado y crear un DataFrame
        df = pd.read_excel(archivo['title'])
        # Agregar el DataFrame al array
        files_dataframes.append(df)
        print("Archivo descargado y DataFrame creado:", archivo['title'])
    except Exception as e:
        print("Error al descargar y crear DataFrame para el archivo:", archivo['title'])
        print(e)

Archivo descargado y DataFrame creado: Estacion_VALLE DE BRAVO_90_dias.xlsx


In [68]:
def limpiar_datos(df):
    columnas_requeridas = ['Fecha Local','Temperatura del Aire (°C)','Precipitación (mm)']
    df = df[columnas_requeridas]
    #Renombrar columnas
    df = df.rename(columns={'Fecha Local' : 'Fecha','Temperatura del Aire (°C)' : 'Temperatura','Precipitación (mm)' : 'Precipitacion'})
    df['Fecha'] = pd.to_datetime(df['Fecha'],errors = "coerce")

    df["Fecha"] = df["Fecha"].dt.date
    # Calcular la temperatura máxima y mínima de cada día
    df_agrupado = df.groupby('Fecha')['Temperatura'].agg(['max', 'min']).reset_index()
    #Calcular el promedio de la precipitación
    df['Precipitacion'].fillna(0 , inplace = True)
    df_agrupadoPre = df.groupby('Fecha')['Precipitacion'].mean().reset_index()
    # Renombrar las columnas
    df_agrupado.columns = ['Fecha', 'TemperaturaMaxima', 'TemperaturaMinima']
    df_agrupado['Precipitacion'] = df_agrupadoPre['Precipitacion']
    df_agrupado = df_agrupado.dropna(subset = ['TemperaturaMaxima','TemperaturaMinima'], how = "all")
    return df_agrupado

In [69]:
with ThreadPoolExecutor(max_workers=4) as executor:
    # Ejecuta la función de limpieza para cada archivo
    resultados = list(executor.map(limpiar_datos, files_dataframes))

df_final = pd.concat(resultados, ignore_index = True)

In [70]:
# df_final = limpiar_datos(files_dataframes)
df_final.to_csv('resultados_Prueba.csv', index = False)